In [35]:
import torch 
from torch.utils.data import DataLoader, Dataset
import sklearn
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline

## Testing on the Pistacio_Image_Dataset

### Dataset

In [51]:

class CustomDataset(Dataset):
    def __init__(self, transforms=None):
        
        base_dir = os.path.dirname(os.path.abspath(__name__))
        image_dir = os.path.join("Pistachio_Image_Dataset", "Pistachio_Image_Dataset")
        
        class_translation = {0: "Kirmizi", 1: "siirt"}
        class_dir_names = {0: "Kirmizi_Pistachio", 1: "Siirt_Pistachio"}
        
        kirmizi_abs_files = [os.path.join(image_dir, class_dir_names[0], file) for file in os.listdir(os.path.join(image_dir, class_dir_names[0]))]
        siirt_abs_files = [os.path.join(image_dir, class_dir_names[1], file) for file in os.listdir(os.path.join(image_dir, class_dir_names[1]))]

        self.image_files = kirmizi_abs_files + siirt_abs_files
        self.classes = np.full(len(kirmizi_abs_files), 0).tolist() + np.full(len(siirt_abs_files), 1).tolist()
        self.transforms = transforms

    def __len__(self):
        return len(self.classes)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.image_files[idx]
        img_class = self.classes[idx]
        
        image = cv2.imread(img_path)
        sample = {"image": image, "class": img_class}

        if self.transforms is not None:
            smaple = self.transforms(sample)

        return sample
        
ds = CustomDataset()

In [52]:
len(ds)

2148

In [53]:
ds[0]

{'image': array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 'class': 0}